In [32]:
import torch
import torchvision
from torch import nn
from torchvision import transforms, models, datasets
from torchinfo import summary

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

In [33]:
torch.manual_seed(2929)

In [34]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...") 
        zip_ref.extractall(image_path)

    # Remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

data\pizza_steak_sushi directory exists.


In [35]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'

In [36]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
#weights
base_transform = weights.transforms()
base_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [37]:
train_dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(train_dir, transform=base_transform),
    batch_size=32,
    shuffle=True,
    num_workers=4,
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(test_dir, transform=base_transform),
    batch_size=32,
    shuffle=False,
    num_workers=4,
)
# Check the class names
class_names = train_dataloader.dataset.classes
print(class_names)

['pizza', 'steak', 'sushi']


In [38]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)
#model

In [39]:
summary(model=model,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'],
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

Freezing base model, change output layer

In [40]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False

In [41]:
output_shape = len(class_names)

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape,
                    bias=True)
    )

In [42]:
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)
#model

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

define train step, test step and combine them

In [43]:
def train_model(model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                criterion: torch.nn.Module, 
                optimizer: torch.optim.Optimizer,
                device: torch.device) -> Tuple[float, float]:
    
    # Set the model to training mode
    model.train()

    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        #Forward pass
        y_pred = model(X)

        #calculate  and accumulate loss
        loss = criterion(y_pred, y)
        train_loss += loss.item() 

        #Optimizer zero grad
        optimizer.zero_grad()

        #Loss backward
        loss.backward()

        #Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
            
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
        
    return train_loss, train_acc

In [44]:
def test_model(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               criterion: torch.nn.Module,
               device: torch.device) -> Tuple[float, float]:
    #eval mode on
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        #loop through DataLoader batches:
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # Forward pass
            test_pred = model(X)

            # Calculate and accumulate loss
            loss = criterion(test_pred, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy metric across all batches
            y_pred_labels = test_pred.argmax(dim=1)
            test_acc += (y_pred_labels == y).sum().item()/len(y_pred_labels)
    
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [45]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          criterion: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    # Create a dictionary to store the results
    results = {"train_loss": [], 
               "train_acc": [],
               "test_loss": [], 
               "test_acc": []
               }

    # Loop through the epochs
    for epoch in tqdm(range(epochs)):
        # Train the model
        train_loss, train_acc = train_model(model=model,
                                            dataloader=train_dataloader,
                                            criterion=criterion,
                                            optimizer=optimizer,
                                            device=device)
        
        # Test the model
        test_loss, test_acc = test_model(model=model,
                                         dataloader=test_dataloader,
                                         criterion=criterion,
                                         device=device)

        # Print the results
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

        # Append the results to the dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

In [46]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                criterion=criterion,
                optimizer=optimizer,
                epochs=5,
                device="cuda" if torch.cuda.is_available() else "cpu")

 20%|██        | 1/5 [00:32<02:10, 32.75s/it]

Epoch 1/5
Train loss: 0.9936, Train accuracy: 0.6328
Test loss: 0.8519, Test accuracy: 0.7443


 40%|████      | 2/5 [00:59<01:27, 29.21s/it]

Epoch 2/5
Train loss: 0.8827, Train accuracy: 0.6523
Test loss: 0.7672, Test accuracy: 0.7443


 60%|██████    | 3/5 [01:26<00:56, 28.14s/it]

Epoch 3/5
Train loss: 0.7240, Train accuracy: 0.8633
Test loss: 0.6917, Test accuracy: 0.8258


 80%|████████  | 4/5 [01:53<00:27, 27.69s/it]

Epoch 4/5
Train loss: 0.6837, Train accuracy: 0.7852
Test loss: 0.6258, Test accuracy: 0.8362


100%|██████████| 5/5 [02:20<00:00, 28.18s/it]

Epoch 5/5
Train loss: 0.6192, Train accuracy: 0.8164
Test loss: 0.6056, Test accuracy: 0.8163


pred function

In [ ]:
from PIL import Image

def pred_img(model: torch.nn.Module,
             image_path: str,
             class_names: List[str],
             transform: torchvision.transforms = None,
             device: torch.device = "cuda" if torch.cuda.is_available() else "cpu"
             ) -> None:
    
    # Load the image
    img = Image.open(image_path)
    # Transform the image
    if transform is not None:
        imgage_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    #predict
    model.to(device)
    model.eval()
    with torch.inference_mode():
        transformed_image = image_transform(img).unsqueeze(0)
        target_image_pred = model(transformed_image.to(device))
        pred_probs = torch.softmax(target_image_pred, dim=1)
        pred_label = torch.argmax(pred_probs, dim=1)
        class_name = class_names[pred_label.item()]
        print(f"Predicted class: {class_name}")

In [ ]:
test_img_path = list(Path('./test_img').glob('*.jpg'))
for img_path in test_img_path:
    pred_img(model=model,
             image_path=img_path,
             class_names=class_names,
             transform=None,
             device="cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'img' is not defined